# Upscaling one raster dataset based on another

### Statement of problem

We have two raster timeseries datasets from different sources, nominally representing the same variable. One is at relatively high spatial resolution but is only annual in temporal resolution. The other is at coarse spatial resolution but is available at monthly (or finer) temporal resolution.

We wish to combine the information from these datasets to interpolate higher-resolution monthly data. We use the principle:

    HiRes(y,m) = LowRes(y,m) * HiRes(y) / sum(LowRes(y,1)..LowRes(y,12))

or 

    HiRes(y,m) = LowRes(y,m) * HiRes(y) / mean(LowRes(y,1)..LowRes(y,12))

This example is taken from [this paper](https://www.thelancet.com/journals/lanplh/article/PIIS2542-5196(19)30047-6/fulltext#supplementaryMaterial) (Eq2 of the supplementary material). Here the "low resolution" data are PM<sub>2.5</sub> rasters derived from MERRA-2, and the "high resolution" data are PM<sub>2.5</sub> rasters obtained from [this site](http://fizz.phys.dal.ca/~atmos/martin/?page_id=140). We would like to create monthly PM<sub>2.5</sub> rasters at the spatial resolution (0.01 degrees) of the annual dataset.

(*However, the equation referenced uses the annual sum of the low-res data, but this seems to be wrong for a quantity such as PM2.5 where a mean is surely the correct approach. (The high-resolution annual data are not a "sum" of all the months, they are an annual average). Sum would be appropriate for e.g. precipitation.*)

The basic challenge is thus to overlay and perform an array-wise calculation between two rasters which are of different sizes (resolution and extent). We will use rasterio to create a virtual raster from the low resolution rasters, which is warped and reprojected to the same extent and resolution as the high resolution raster.

In [13]:
import rasterio as rio
from rasterio.crs import CRS
from rasterio.enums import Resampling
from rasterio import shutil as rio_shutil
from rasterio.vrt import WarpedVRT
import affine
import numexpr as ne


In [2]:
import os
import glob

In [3]:
import numpy as np

In [4]:
from matplotlib import pyplot

In [5]:
LOWRES_MONTHLY_FOLDER = r'D:\InformalCities\Data\AirPollution\MERRA-2_Subset\PM_2p5_calc\monthly'
HIRES_ANNUAL_FOLDER = r'D:\InformalCities\Data\AirPollution\PM25\UnCorrected'
LOWRES_ANNUAL_AVG_FOLDER = os.path.join(os.path.dirname(LOWRES_MONTHLY_FOLDER), 'annual_mean')
#acag_downscaled_output = r'F:\InformalCities\Data\AirPollution\PM25\temporal_downscale_python'
acag_downscaled_output = r'C:\temp\acag_downscaled'

In [ ]:
merra_monthlies = glob.glob(os.path.join(LOWRES_MONTHLY_FOLDER, "*.tif"))
merra_monthlies


In [ ]:
acag_annuals = glob.glob(os.path.join(HIRES_ANNUAL_FOLDER, "*.tif"))
acag_annuals

In [22]:
years = range(2000,2019)

In [9]:
with rio.open(merra_monthlies[0]) as merra_sample:
    merra_bounds = merra_sample.bounds
    merra_crs = merra_sample.crs
    merra_shape = merra_sample.shape

    

Pre-process by summarising the merra monthly files to annual averages once, before upsampling.

We also adjust the units at this stage as the MERRA-2 data are in kg/m3 and the ACAG data are in ug/m3

In [ ]:
merra_annuals = {}
for year in years:
    monthlies = glob.glob(os.path.join(LOWRES_MONTHLY_FOLDER, f'MERRA2_Pm2p5.{year}.*'))
    year_files = np.ma.empty(shape=(len(monthlies), merra_shape[0], merra_shape[1]))
    profile = None
    for i, f in enumerate(monthlies):
        with rio.open(f) as src:
            year_files[i]= src.read(1, masked=True)
            if not profile:
                profile = src.profile
    # average of months, convert the units to match acag as well (ug/m3 rather than kg/m3)
    year_avg = year_files.mean(axis=0) * 1e9
    with rio.Env():
        annual_path = os.path.join(LOWRES_ANNUAL_AVG_FOLDER, f'MERRA2_Pm2p5_AnnualAverage.{year}.tif')
        with rio.open(annual_path, 'w', **profile) as dst:
            dst.write(year_avg, 1)
        merra_annuals[year] = annual_path                  

### Perform upscaling calculation between ACAG and MERRA-2 data

#### align merra-2 to acag using in-memory vrt
We wish to do a vectorised / array operation between the data in the (0.1 degree) annual ACAG data and the (0.625 * 0.5) degree monthly MERRA-2 data, at the 0.1 degree resolution. So, we have to resample the MERRA-2 data into the same shape array.

This would happen automatically / implicitly if performing a raster calculation in a GIS (albeit only with nearest neighbour resampling) but in python the code has to be written to do that.

Using rasterio, if the datasets had the same extents but different cellsizes, we could simply read the data specifying the shape and the upsampling would happen automatically, e.g.

```python
with rio.open(merra_monthlies[0]) as src:
    data2 = src.read(
    out_shape=(1,acag_height, acag_width),
    resampling=Resampling.bilinear)
    tf2 = src.transform* src.transform.scale((src.width / data2.shape[-1]), (src.height / data2.shape[-2]))
```

However as they don't have the same extent OR resolution, we want to use a vrt to re-map it
 https://rasterio.readthedocs.io/en/latest/topics/virtual-warping.html

Instead of nearest-neighbour we will use a bilinear interpolation of the coarse data to avoid a blocky / tiling effect in the upscaled data along the coarse cell boundaries. 

The following code will for each year:
 - read the annual high-resolution ACAG raster
 - read the (previously-created) annual average MERRA-2 raster, upsampling it on read using bilinear interpolation to the extent and shape matching the ACAG raster (arrays will be of same dimensions)
 - calculate the correction factor for this year
 - For each month in the year:
   - Read the month's MERRA-2 raster, upsampling it in the same way as above
   - Multiply it to get in same units as the ACAG-2 and merra annuals
   - Multiply it by the year's correction factor raster
   - Mask where either the ACAG or MERRA rasters were masked
   
We use numexpr for efficient multithreaded evaluation of the array calculations, and use multithreaded compression for efficient creation of the output geotiffs.


In [ ]:
for year in years:
    acag_raster_fn = glob.glob(os.path.join(HIRES_ANNUAL_FOLDER, 
                                            f'ACAG_PM25*{year}.tif'))[0]
    with rio.open(acag_raster_fn) as acag_src:
        profile = acag_src.profile
        profile.update(dtype=rio.float64)
        profile.update(num_threads=8)
        left, bottom, right, top = acag_src.bounds
        acag_height, acag_width = acag_src.shape
        acag_xres = (right-left) / acag_width
        acag_yres = (top-bottom) / acag_height
        acag_transform = affine.Affine(acag_xres, 0.0, left, 0.0, -acag_yres, top)
        acag_crs = acag_src.crs
        vrt_options = {
            'resampling': Resampling.bilinear,
            'crs': acag_crs,
            'transform': acag_transform,
            'height': acag_height,
            'width': acag_width,
            'num_threads': 8
        }
        acag_data = acag_src.read(1, masked=True)
    merra_annual_fn = (os.path.join(LOWRES_ANNUAL_AVG_FOLDER, 
                                    f'MERRA2_Pm2p5_AnnualAverage.{year}.tif'))     
    #merra_annual_fn = merra_annuals[year]
    with rio.open(merra_annual_fn) as merra_src:
        with WarpedVRT(merra_src, **vrt_options) as vrt:
            merra_annual_data = vrt.read(1, masked=True)
    assert(acag_data.shape == merra_annual_data.shape)
    corrector = ne.evaluate("acag_data / merra_annual_data")
    mask = np.logical_or(acag_data.mask, merra_annual_data.mask)
    del(acag_data) # likely to be on the limits for a 16GB RAM machine so get rid asap
    del(merra_annual_data)
    monthlies = glob.glob(os.path.join(LOWRES_MONTHLY_FOLDER, f'MERRA2_Pm2p5.{year}.*'))
    for merra_monthly_fn in monthlies:
        _, t2, m, _ = merra_monthly_fn.split('.')
        outname = os.path.join(acag_downscaled_output, f'ACAG_MERRA_PM2p5_Downscale.{year}.{m}.tif')
        if os.path.exists(outname):
            continue
        with rio.open(merra_monthly_fn) as merra_src:
            with WarpedVRT(merra_src, **vrt_options) as vrt:
                merra_month_data = vrt.read(1, masked=True)
        
        downscaled_data = ne.evaluate("merra_month_data * 1e9 * corrector")
        downscaled_data[np.logical_or(mask, merra_month_data.mask)] = profile['nodata'] 
        del(merra_month_data)
        with rio.open(outname, 'w', **profile) as dst:
            dst.write(downscaled_data, 1)
        